# Explain-CNN-With-GANS: Data Preparation Workflow
This notebook covers the full data preparation pipeline: dataset download, classifier training, GradCAM/CGMA generation, and .npy creation for GAN training.

In [ ]:
# Install required packages
!pip install kagglehub tensorflow opencv-python matplotlib

In [ ]:
import tensorflow as tf
import os
import numpy as np
from tensorflow.keras import applications as ap
import kagglehub
import shutil
import cv2
import matplotlib.pyplot as plt

## 1. Download Food-11 Dataset

In [ ]:
dataset_path = kagglehub.dataset_download("trolukovich/food11-image-dataset")
print("Path to dataset files:", dataset_path)
target_path = "Datasets/Food"
try:
    shutil.move(dataset_path, target_path)
    print(f"Dataset moved to {target_path}")
except Exception as e:
    print(f"Could not move dataset: {e}")

## 2. Train Classifiers (InceptionV3, ResNet50, VGG16)

In [ ]:
models =[ap.InceptionV3, ap.ResNet50, ap.VGG16]
data_paths =['Datasets/Food']
model_paths= ['models/Food']
backbone_names =['InceptionV3', 'ResNet', 'VGG16']
for j, path in enumerate(data_paths):
    for i, backbone_model in enumerate(models):
        gen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1/255., validation_split=0.1)
        train = gen.flow_from_directory(path, target_size=(256,256), batch_size=64, subset='training')
        val = gen.flow_from_directory(path, target_size=(256,256), batch_size=64, subset='validation')
        m = backbone_model(input_shape=(256,256,3), include_top=False)
        m_out = m.output
        glob = tf.keras.layers.GlobalMaxPooling2D()(m_out)
        d1 = tf.keras.layers.Dense(256, activation='relu', kernel_initializer='he_normal')(glob)
        drop = tf.keras.layers.Dropout(0.1)(d1)
        d2 = tf.keras.layers.Dense(128, activation='relu', kernel_initializer='he_normal')(d1)
        out = tf.keras.layers.Dense(len(np.unique(train.classes)), activation='softmax', kernel_initializer='he_normal')(d2)
        model = tf.keras.models.Model(inputs=m.input, outputs=out)
        opt = 'adam' if backbone_names[i]=='mobilenet' else 'sgd'
        print(f'Model = {backbone_names[i]}')
        print(f"Optimizer = {opt}")
        model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
        try:
            os.makedirs(os.path.join(model_paths[j], backbone_names[i]), exist_ok=True)
        except Exception as e:
            print(f"Could not create model directory: {e}")
        callbacks = [tf.keras.callbacks.ModelCheckpoint(monitor='val_loss', filepath=os.path.join(model_paths[j], backbone_names[i], 'Epoch={epoch:02d}- Loss={val_loss:.2f} - val_acc = {val_accuracy:.2f}.h5'))]
        history = model.fit(train, epochs=20, validation_data=val, callbacks=callbacks)

## 3. Generate GradCAM and CGMA Maps

In [ ]:
# Example GradCAM function (simplified)
def GradCam(model, img_array, layer_name):
    grad_model = tf.keras.models.Model([model.inputs], [model.get_layer(layer_name).output, model.output])
    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(img_array)
        loss = predictions[:, tf.argmax(predictions[0])]
    grads = tape.gradient(loss, conv_outputs)[0]
    weights = tf.reduce_mean(grads, axis=(0, 1))
    cam = tf.reduce_sum(tf.multiply(weights, conv_outputs[0]), axis=-1)
    cam = np.maximum(cam, 0)
    cam = cam / np.max(cam)
    cam = cv2.resize(cam.numpy(), (img_array.shape[2], img_array.shape[1]))
    return cam
# You can loop over images and layers to generate and save GradCAM/CGMA maps as needed.

## 4. Create .npy File for GAN Training

In [ ]:
# Example: Combine images, GradCAMs, CGMAs, and labels into a single .npy file
def create_gan_training_npy(image_folder, gradcam_folder, cgma_folder, labels_file, output_npy):
    images = []
    gradcams = []
    cgmas = []
    labels = np.load(labels_file)
    for img_name in os.listdir(image_folder):
        img = cv2.imread(os.path.join(image_folder, img_name))
        images.append(img)
        gradcam = np.load(os.path.join(gradcam_folder, img_name.replace('.jpg', '.npy')))
        gradcams.append(gradcam)
        cgma = np.load(os.path.join(cgma_folder, img_name.replace('.jpg', '.npy')))
        cgmas.append(cgma)
    np.save(output_npy, {'images': images, 'gradcams': gradcams, 'cgmas': cgmas, 'labels': labels})
    print(f'GAN training data saved to {output_npy}')